In [ ]:
import numpy as np
import pandas as pd
import altair as alt
from eisp import proxy_tasks
from sklearn.manifold import TSNE

In [ ]:
csv_path = '../rcpd/rcpd_annotation_fix.csv'
dfy = pd.read_csv(csv_path) 


In [ ]:
prepend_img_path = "../rcpd/images" # do not put the final '/' here
imgname = dfy.iloc[:, 1].values
imgname = list(map(lambda x: prepend_img_path + x, imgname))
colors = dfy.iloc[:, -1].values


In [ ]:
print(imgname)

In [ ]:
feature_path = '../features_all/' # If testing with Dino, change to '../features/'

features: proxy_tasks.FeatureVectors = proxy_tasks.FeatureVectors.from_files(feature_path)

In [ ]:
FEATURE_TO_PLOT = 'ALL'

features_to_plot = []
if FEATURE_TO_PLOT == 'ALL':
    feature_dict = features.get_all_features()
    features_to_plot = np.concatenate(list(feature_dict.values()), axis=1)
else:
    features_to_plot = features.get_feature(FEATURE_TO_PLOT)
print(features_to_plot.shape)

In [ ]:
tsne_features = TSNE(n_components=2, random_state=42).fit_transform(features_to_plot)

tsne_df = pd.DataFrame(data=tsne_features, columns=['x', 'y'])
tsne_df['image'] = imgname
tsne_df['color'] = colors

In [ ]:
alt.Chart(tsne_df).mark_circle(size=30).encode(
    x="x",
    y="y",
    color=alt.Color('color:N', scale=alt.Scale(scheme='lightgreyred')),
    tooltip=['image']
).properties(
    width=2400,
    height=1800
).interactive().configure_view(strokeWidth=0).save("tsne_visualization.html")


In [ ]:
chart = alt.Chart(tsne_df[::2]).mark_image(
    width=12*4,
    height=12*4).encode(
    x="x",
    y="y",
    url="image",
    ).properties(
    width=800*4,
    height=600*4
)

In [ ]:
chart